In [1]:
from keras.applications import VGG16
conv_base = VGG16(weights='imagenet',
 include_top=False,
 input_shape=(150, 150, 3))
# Здесь конструктору передаются три аргумента:
# 1 Аргумент weights определяет источник весов для инициализации модели.
# 2 Аргумент include_top определяет необходимость подключения к сети полносвязного классификатора. По умолчанию этот полносвязный
#классификатор соответствует 1000 классов из ImageNet. Так как мы намереваемся использовать свой полносвязный классификатор
#(только с двумя классами: cat и dog), мы не будем подключать его.
# 3 Аргумент input_shape определяет форму тензоров с изображениями, которые будут подаваться на вход сети. 
# Это необязательный аргумент: если опустить его, сеть сможет обрабатывать изображения любого размера.
conv_base.summary()

Using TensorFlow backend.
W0808 16:00:48.865005  4864 deprecation_wrapper.py:119] From C:\Users\Batman\Anaconda3\envs\tensor\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0808 16:00:48.919858  4864 deprecation_wrapper.py:119] From C:\Users\Batman\Anaconda3\envs\tensor\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0808 16:00:48.937810  4864 deprecation_wrapper.py:119] From C:\Users\Batman\Anaconda3\envs\tensor\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0808 16:00:48.986713  4864 deprecation_wrapper.py:119] From C:\Users\Batman\Anaconda3\envs\tensor\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [2]:
#Быстрое выделение признаков без расширения данных
#Выделение признаков с использованием предварительно обученной сверточной основы
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
base_dir = r'D:\Batman\Pictures\cats_vs_dogs\dataset'#основной каталог
train_dir = r'D:\Batman\Pictures\cats_vs_dogs\dataset\training_set'#каталог с обучающими данными
validation_dir = r'D:\Batman\Pictures\cats_vs_dogs\dataset\validation_set'
test_dir = r'D:\Batman\Pictures\cats_vs_dogs\dataset\test_set'
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20
def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
     directory,
     target_size=(150, 150),
     batch_size=batch_size,
     class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels
train_features, train_labels = extract_features(train_dir, 6800)
validation_features, validation_labels = extract_features(validation_dir, 1200)
test_features, test_labels = extract_features(test_dir, 2000)
train_features = np.reshape(train_features, (6800, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1200, 4 * 4 * 512))
test_features = np.reshape(test_features, (2000, 4 * 4 * 512))

Found 6800 images belonging to 2 classes.


UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node block1_conv1/convolution}}]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node block1_conv1/convolution}}]]
	 [[block5_pool/MaxPool/_159]]
0 successful operations.
0 derived errors ignored.

In [ ]:
#Определение и обучение полносвязного классификатора
from keras import models
from keras import layers
from keras import optimizers
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
 loss='binary_crossentropy',
 metrics=['acc'])
history = model.fit(train_features, train_labels,
 epochs=5,
 batch_size=20,
 validation_data=(validation_features, validation_labels))

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
#Выделение признаков с расширением данных
from keras import models
from keras import layers
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
 print('This is the number of trainable weights '
 'before freezing the conv base:', len(model.trainable_weights))
conv_base.trainable = False# заморозка слоев, предотвращает изменение весовых коэфиценов
#если этого не сделать, тогда представления, полученные сверточной основой, могут изменится в процессе обучения на новых данных
print('This is the number of trainable weights '
 'after freezing the conv base:', len(model.trainable_weights))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
train_datagen = ImageDataGenerator(
 rescale=1./255,
 rotation_range=40,
 width_shift_range=0.2,
 height_shift_range=0.2,
 shear_range=0.2,
 zoom_range=0.2,
 horizontal_flip=True,
 fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255) #Проверочнве данные не требуется рассширять!
train_generator = train_datagen.flow_from_directory(
 train_dir, 
 target_size=(150, 150),#Приведение всех изображений к размеру 150 × 150
 batch_size=20,
 class_mode='binary')
validation_generator = test_datagen.flow_from_directory(
 validation_dir,
 target_size=(150, 150),
 batch_size=20,
 class_mode='binary')
model.compile(loss='binary_crossentropy',
 optimizer=optimizers.RMSprop(lr=2e-5),
 metrics=['acc'])
history = model.fit_generator(
 train_generator,
 steps_per_epoch=100,
 epochs=30,
 validation_data=validation_generator,
 validation_steps=50)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
#дообучение модели (проверить всё)
#дообучем несколько верхних слоёв так как они хранят более частные признаки,
#чем верхние слои, которые хранят более обобщенные признаки
#замораживание всех слоев, кроме заданных
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
if layer.name == 'block5_conv1':
set_trainable = True
if set_trainable:
layer.trainable = True
else:
layer.trainable = False
#дообучение модели
model.compile(loss='binary_crossentropy',
optimizer=optimizers.RMSprop(lr=1e-5),
metrics=['acc'])
history = model.fit_generator(
train_generator,
steps_per_epoch=100,
epochs=100,
validation_data=validation_generator,
validation_steps=50)
#сглаживание грвфиков
def smooth_curve(points, factor=0.8):
smoothed_points = []
for point in points:
if smoothed_points:
previous = smoothed_points[-1]
smoothed_points.append(previous * factor + point * (1 - factor))
else:
smoothed_points.append(point)
return smoothed_points
plt.plot(epochs,
smooth_curve(acc), 'bo', label='Smoothed training acc')
plt.plot(epochs,
smooth_curve(val_acc), 'b', label='Smoothed validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs,
smooth_curve(loss), 'bo', label='Smoothed training loss')
plt.plot(epochs,
smooth_curve(val_loss), 'b', label='Smoothed validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()
#оценка моели на контрольных данных
test_generator = test_datagen.flow_from_directory(
test_dir,
target_size=(150, 150),
batch_size=20,
class_mode='binary')
test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)